# Extracción de Texto de Documentos Legales

## Objetivo
Este notebook extrae artículos individuales de documentos legales (PDFs y DOCX) y los guarda en un formato estructurado.

## Salida
- **Archivo**: Nombre del archivo sin extensión
- **Articulo**: Número/nombre del artículo  
- **Texto**: Contenido del artículo

## Funcionalidades principales:

### Limpieza de contenido:
1. **`separar_transitorios`**: Separa artículos transitorios del cuerpo principal
2. **`quitar_encabezados_conservador`**: Elimina encabezados estructurales (CAPÍTULO, TÍTULO, etc.)
3. **`limpiar_contenido_articulo`**: Limpia encabezados residuales DENTRO de cada artículo
   - Protege líneas entre paréntesis (ej: DEROGADO, REFORMADO)
   - Protege palabras clave de estado

### Extracción de PDFs:
- Opción de recortar encabezados/pies de página antes de extraer texto
- Usa PyMuPDF para crear un PDF temporal sin headers/footers
- Luego extrae texto limpio con Apache Tika

**Nota**: El matching con el catálogo de leyes y la creación de IDs se hace en el siguiente paso (`hash_creation.ipynb`)


In [41]:
import os
import re
import pandas as pd
from unicodedata import normalize
from tika import parser
from docx import Document
import tempfile
import fitz  # PyMuPDF


In [42]:
def normaliza_texto(s: str) -> str:
    """
    Normalización básica de texto.
    Unifica espacios especiales y normaliza unicode.
    """
    if not s: 
        return s
    s = normalize("NFKC", s)
    # NBSP y otros espacios duros
    s = s.replace("\u00A0"," ").replace("\u2007"," ").replace("\u202F"," ")
    # opcional: unificar "°" -> "º"
    s = s.replace("°", "º")
    return s


In [43]:
# Detecta encabezados principales de "Transitorios" en línea aislada
RE_TRANS_HEADER = re.compile(
    r'(?im)^\s*(?:DISPOSICIONES\s+TRANSITORIAS|ART[IÍ]CULOS?\s+TRANSITORIOS?|TRANSITORIOS?)\s*[:\-–—]?\s*$'
)

def separar_transitorios(texto: str) -> tuple:
    """
    Separa el texto en dos partes: texto principal y transitorios.
    
    Returns:
        tuple: (texto_principal, texto_transitorios)
        - texto_principal: texto antes del encabezado TRANSITORIOS
        - texto_transitorios: texto después del encabezado TRANSITORIOS (sin el encabezado)
                              None si no hay transitorios
    """
    if not texto:
        return texto, None

    # Buscar encabezado de TRANSITORIOS
    m = RE_TRANS_HEADER.search(texto)
    if m:
        # Separar en texto principal y transitorios
        texto_principal = texto[:m.start()].rstrip()
        # Capturar todo después del encabezado (sin incluir el encabezado mismo)
        texto_transitorios = texto[m.end():].strip()
        return texto_principal, texto_transitorios
    
    # No hay transitorios
    return texto, None


In [44]:
# Patrón mejorado: Encabezados estructurales con subtítulo opcional
RE_ESTRUCTURA_MEJORADO = re.compile(r'''
(?:^|\n)                                    # inicio o nueva línea
[ \t]*                                      # espacios/tabs opcionales
(?:                                         # Palabra clave del encabezado
  T[ÍI]TULO | TITULO |
  CAP[ÍI]TULO | CAPITULO |
  SECCI[ÓO]N | SECCION |
  LIBRO
)
\s+
(?:                                         # N° romano, arábigo u ordinal
  [IVXLCDM]+ | \d+ |
  PRIMERO|SEGUNDO|TERCERO|CUARTO|QUINTO|SEXTO|
  S[EÉ]PTIMO|OCTAVO|NOVENO|D[EÉ]CIMO|UND[EÉ]CIMO|DUOD[EÉ]CIMO
)
(?:[^\n]{0,100})?                           # resto de la línea del encabezado
\s*\n
(?:                                         # Subtítulo opcional (siguiente línea)
  [ \t]*
  [A-ZÁÉÍÓÚÑ][A-ZÁÉÍÓÚÑ0-9\s\.\,\-–—:/()']{5,120}  # línea en mayúsculas (subtítulo)
  \s*\n
)?
''', re.MULTILINE | re.VERBOSE)

def quitar_encabezados_conservador(texto: str) -> str:
    """
    Versión MEJORADA: Elimina encabezados estructurales con sus subtítulos.
    - Detecta CAPÍTULO/TÍTULO/SECCIÓN + número
    - Captura subtítulo opcional en la siguiente línea (si está en MAYÚSCULAS)
    - Protege contenido que no sea encabezado
    """
    if not texto:
        return texto

    # Elimina encabezados y sus subtítulos
    t = RE_ESTRUCTURA_MEJORADO.sub("\n", texto)
    
    # Limpia saltos de línea excesivos
    t = re.sub(r"\n{3,}", "\n\n", t).strip()
    return t


In [45]:
def limpiar_contenido_articulo(contenido: str) -> str:
    """
    Limpia el contenido DENTRO de un artículo ya extraído.
    Elimina encabezados residuales, normaliza espacios y saltos de línea.
    """
    if not contenido:
        return contenido
    
    # 1. Eliminar líneas completas en MAYÚSCULAS (probables encabezados residuales)
    lineas = contenido.split('\n')
    lineas_limpias = []
    
    for linea in lineas:
        linea_stripped = linea.strip()
        
        # Skip líneas vacías (las conservamos)
        if not linea_stripped:
            lineas_limpias.append(linea)
            continue
        
        # Skip líneas muy cortas o muy largas
        if len(linea_stripped) < 5 or len(linea_stripped) > 150:
            lineas_limpias.append(linea)
            continue
        
        # PROTEGER líneas entre paréntesis (anotaciones como DEROGADO, REFORMADO, etc.)
        if linea_stripped.startswith('('):
            lineas_limpias.append(linea)
            continue
        
        # PROTEGER líneas con palabras clave de estado de artículos
        palabras_estado = ['DEROGADO', 'REFORMADO', 'ADICIONADO', 'MODIFICADO', 
                          'ABROGADO', 'FE DE ERRATAS', 'G.O.', 'GACETA OFICIAL']
        if any(palabra in linea_stripped.upper() for palabra in palabras_estado):
            lineas_limpias.append(linea)
            continue
        
        # Contar letras mayúsculas vs total de letras
        letras_mayus = sum(1 for c in linea_stripped if c.isupper())
        letras_minus = sum(1 for c in linea_stripped if c.islower())
        total_letras = letras_mayus + letras_minus
        
        # Si tiene letras y al menos 80% son mayúsculas -> probable encabezado
        if total_letras >= 3 and (letras_mayus / total_letras) >= 0.80:
            # Verificación adicional: no debe empezar con números romanos sueltos
            # (para proteger fracciones como "I. TEXTO", "II. TEXTO")
            if not re.match(r'^[IVXLCDM]{1,4}[\.\)]\s', linea_stripped):
                # Es un encabezado en MAYÚSCULAS, lo omitimos
                continue
        
        # Conservar la línea
        lineas_limpias.append(linea)
    
    contenido = '\n'.join(lineas_limpias)
    
    # 2. Normaliza espacios múltiples
    contenido = re.sub(r'[ \t]+', ' ', contenido)
    
    # 3. Normaliza saltos de línea excesivos
    contenido = re.sub(r'\n{3,}', '\n\n', contenido)
    
    return contenido.strip()


In [46]:
def extraer_texto_docx(ruta_docx):
    """ Extrae el texto de un archivo .docx """
    try:
        doc = Document(ruta_docx)
        return "\n".join([p.text for p in doc.paragraphs]).strip()
    except Exception as e:
        print(f"Error al procesar '{ruta_docx}': {e}")
        return None

def extraer_texto_pdf(ruta_pdf):
    """ Extrae el texto de un PDF usando Apache Tika """
    try:
        raw = parser.from_file(ruta_pdf)  
        return raw.get("content", "").strip()
    except Exception as e:
        print(f"Error al procesar '{ruta_pdf}': {e}")
        return None


In [47]:
def extraer_articulos(texto):
    """ 
    Extrae menciones completas de artículos (ej: "Artículo 23 Bis") de documentos 
    legales de la Ciudad de México. Solo captura menciones que inician con A mayúscula
    al principio de línea para evitar menciones en medio del texto.
    """
    if not texto:
        return []

    # PATRÓN COMPLEJO para artículos legales
    # Captura la mención COMPLETA: "Artículo 23 Bis" (no solo "23 Bis")
    patron = r'''
        (?:^|\n)\s*                          # Inicio de línea o nueva línea
        (                                     # GRUPO 1: Mención COMPLETA del artículo
            (?:ART[ÍI]CULO|Art[íi]culo|ARTICULO|Articulo|Art\.)  # Palabra "Artículo"
            \s+                               # Espacios obligatorios
            \d+                               # Número base (ej: 23)
            (?:\s*[°ºªo])?                    # Símbolo de grado opcional (incluye 'o')
            (?:                               # Sufijos opcionales - inicio grupo
                \s+(?:B|T|Q|S|O|D)\s+\d+      # Abreviatura + número OBLIGATORIO (B 1, T 2, etc.)
                |
                \s+(?:B|T|Q|S|O|D)(?!\s+\d)   # Abreviatura SIN número después (155 B, sin fracción)
                |
                \s+[A-Z](?![a-z])             # Espacio + letra MAYÚSCULA (ej: 272 A, 272 C)
                |
                \s*[-–—]\s*[A-Z]              # Guión + letra (ej: 8o-A, 8o-B)
                |
                \s*[-–—]\s*\d+                # Guión + número (ej: 23-1, 23-2)
                |
                \s+(?:BIS|Bis|bis|TER|Ter|ter|QU[ÁA]TER|Qu[áa]ter|quater|
                      QUINTUS|Quintus|quintus|QUINQUIES|Quinquies|quinquies|QUINTIES|Quinties|quinties|
                      SEXTUS|Sextus|sextus|SEXIES|Sexies|sexies|SEXTIES|Sexties|sexties|
                      SEPTIMUS|Septimus|septimus|SEPTIES|Septies|septies|SEPTIESEP|Septiesep|septiesep|
                      OCTAVUS|Octavus|octavus|OCTIES|Octies|octies|
                      NON[IÍ]ES|Non[ií]es|non[ií]es|NOVIES|Novies|novies|
                      DEC[IÍ]ES|Dec[ií]es|dec[ií]es|DECIMUS|Decimus|decimus|
                      UNDECIES|Undecies|undecies|DUODECIES|Duodecies|duodecies)\s+\d+  # Ordinal + número
                |
                \s+(?:BIS|Bis|bis|TER|Ter|ter|QU[ÁA]TER|Qu[áa]ter|quater|
                      QUINTUS|Quintus|quintus|QUINQUIES|Quinquies|quinquies|QUINTIES|Quinties|quinties|
                      SEXTUS|Sextus|sextus|SEXIES|Sexies|sexies|SEXTIES|Sexties|sexties|
                      SEPTIMUS|Septimus|septimus|SEPTIES|Septies|septies|SEPTIESEP|Septiesep|septiesep|
                      OCTAVUS|Octavus|octavus|OCTIES|Octies|octies|
                      NON[IÍ]ES|Non[ií]es|non[ií]es|NOVIES|Novies|novies|
                      DEC[IÍ]ES|Dec[ií]es|dec[ií]es|DECIMUS|Decimus|decimus|
                      UNDECIES|Undecies|undecies|DUODECIES|Duodecies|duodecies)  # Ordinal sin número
            )?                                # Sufijos opcionales - fin grupo (OPCIONAL)
            \s*                               # Espacios opcionales
            [.:\-–—°]?                       # Separadores opcionales
            \s*                               # Espacios opcionales
            [-–—]?                           # Guión adicional opcional
        )
        \s*                                   # Espacios después de la mención
        (?=\S)                                # Lookahead: debe seguir contenido
        (.*?)                                 # GRUPO 2: Contenido del artículo (no greedy)
        (?=                                   # Lookahead: termina cuando encuentra:
            (?:\n\s*(?:ART[ÍI]CULO|Art[íi]culo|ARTICULO|Articulo|Art\.)\s+\d+)  # Siguiente artículo
            |
            (?:\n\s*(?:CAP[ÍI]TULO|Cap[íi]tulo|CAPITULO|Capitulo|TÍTULO|T[ií]tulo|TITULO))  # Nuevo capítulo/título
            |
            (?:\n\s*(?:TRANSITORIOS?|Transitorios?))  # Artículos transitorios
            |
            \Z                                # Fin del texto
        )
    '''

    articulos = []
    matches = re.finditer(patron, texto, re.MULTILINE | re.DOTALL | re.VERBOSE)
    
    for match in matches:
        mencion_completa = match.group(1).strip()  # "Artículo 23 Bis"
        contenido = match.group(2).strip() if match.group(2) else ""
        
        # Verifica que la mención inicie con A mayúscula
        if not mencion_completa[0].isupper():
            continue
        
        # Limpia el contenido del artículo
        contenido = limpiar_contenido_articulo(contenido)
        
        # Normaliza la mención completa
        mencion_normalizada = normalizar_mencion_articulo(mencion_completa)
        
        articulos.append({
            "Articulo": mencion_normalizada,
            "Articulo_Original": mencion_completa,
            "Texto": contenido
        })

    # Valida la secuencia de artículos
    articulos_validados = validar_secuencia_articulos(articulos)
    
    return articulos_validados


def normalizar_mencion_articulo(mencion):
    """
    Normaliza la mención completa del artículo.
    Ejemplos: 
    - 'ARTICULO 23 BIS' -> 'Artículo 23 Bis'
    - 'Artículo 155 B' -> 'Artículo 155 Bis'
    - 'Artículo 155 B 1' -> 'Artículo 155 Bis 1'
    - 'Art. 23-1' -> 'Artículo 23-1'
    """
    mencion = mencion.strip()
    
    # Normaliza la palabra "Artículo"
    mencion = re.sub(r'^(?:ART[ÍI]CULO|ARTICULO|Art\.)', 'Artículo', mencion, flags=re.IGNORECASE)
    
    # Expande abreviaturas de ordinales (B → Bis, T → Ter, etc.)
    # Captura también el número opcional después de la abreviatura
    mencion = re.sub(r'\b B(\s+\d+)?\b', r' Bis\1', mencion)
    mencion = re.sub(r'\b T(\s+\d+)?\b', r' Ter\1', mencion)
    mencion = re.sub(r'\b Q(\s+\d+)?\b', r' Quáter\1', mencion)
    mencion = re.sub(r'\b S(\s+\d+)?\b', r' Sexies\1', mencion)
    mencion = re.sub(r'\b O(\s+\d+)?\b', r' Octies\1', mencion)
    mencion = re.sub(r'\b D(\s+\d+)?\b', r' Decies\1', mencion)
    
    # Normaliza ordinales latinos a formato estándar
    ordinales = {
        'bis': 'Bis', 'ter': 'Ter', 'quater': 'Quáter', 'quáter': 'Quáter',
        'quintus': 'Quintus', 'quinquies': 'Quinquies', 'quinties': 'Quinties',
        'sextus': 'Sextus', 'sexies': 'Sexies', 'sexties': 'Sexties',
        'septimus': 'Septimus', 'septies': 'Septies', 'septiesep': 'Septies',
        'octavus': 'Octavus', 'octies': 'Octies',
        'nonies': 'Nonies', 'noníes': 'Nonies', 'novies': 'Novies',
        'decies': 'Decies', 'decíes': 'Decies', 'decimus': 'Decimus',
        'undecies': 'Undecies', 'duodecies': 'Duodecies'
    }
    
    for ordinal_min, ordinal_norm in ordinales.items():
        mencion = re.sub(r'\b' + ordinal_min + r'\b', ordinal_norm, mencion, flags=re.IGNORECASE)
    
    # Limpia espacios múltiples
    mencion = re.sub(r'\s+', ' ', mencion)
    
    # Elimina puntos y guiones finales innecesarios
    mencion = re.sub(r'[\.\-–—]+\s*$', '', mencion)
    
    return mencion


def parsear_articulo(mencion):
    """
    Extrae los componentes de un artículo para comparación.
    Retorna: (numero_base, sufijo_ordinal, numero_ordinal, fraccion, letra_sufijo)
    
    Ejemplos:
    - "Artículo 449" → (449, None, None, None, None)
    - "Artículo 449 Bis" → (449, "Bis", None, None, None)
    - "Artículo 449 Bis 2" → (449, "Bis", 2, None, None)
    - "Artículo 498 Bis 6" → (498, "Bis", 6, None, None)
    - "Artículo 23-1" → (23, None, None, 1, None)
    - "Artículo 8o-A" → (8, None, None, None, "A")
    - "Artículo 272 A" → (272, None, None, None, "A")
    - "Artículo 155 Bis" → (155, "Bis", None, None, None)
    - "Artículo 155 Bis 1" → (155, "Bis", 1, None, None)
    """
    # Extrae número base (incluyendo símbolo de grado opcional)
    match_base = re.search(r'Artículo\s+(\d+)[°ºªo]?', mencion)
    if not match_base:
        return None
    
    numero_base = int(match_base.group(1))
    
    # Extrae sufijo ordinal - LISTA COMPLETA
    ordinales_pattern = r'\b(BIS|Bis|bis|TER|Ter|ter|QU[ÁA]TER|Qu[áa]ter|quater|' \
                       r'QUINTUS|Quintus|quintus|QUINQUIES|Quinquies|quinquies|QUINTIES|Quinties|quinties|' \
                       r'SEXTUS|Sextus|sextus|SEXIES|Sexies|sexies|SEXTIES|Sexties|sexties|' \
                       r'SEPTIMUS|Septimus|septimus|SEPTIES|Septies|septies|SEPTIESEP|Septiesep|septiesep|' \
                       r'OCTAVUS|Octavus|octavus|OCTIES|Octies|octies|' \
                       r'NON[IÍ]ES|Non[ií]es|non[ií]es|NOVIES|Novies|novies|' \
                       r'DEC[IÍ]ES|Dec[ií]es|dec[ií]es|DECIMUS|Decimus|decimus|' \
                       r'UNDECIES|Undecies|undecies|DUODECIES|Duodecies|duodecies)\b'
    
    match_ordinal = re.search(ordinales_pattern, mencion, re.IGNORECASE)
    sufijo_ordinal = match_ordinal.group(1) if match_ordinal else None
    
    # Normaliza el sufijo ordinal a formato estándar si existe
    if sufijo_ordinal:
        ordinales_normalizacion = {
            'bis': 'Bis', 'ter': 'Ter', 'quater': 'Quáter', 'quáter': 'Quáter',
            'quintus': 'Quintus', 'quinquies': 'Quinquies', 'quinties': 'Quinties',
            'sextus': 'Sextus', 'sexies': 'Sexies', 'sexties': 'Sexties',
            'septimus': 'Septimus', 'septies': 'Septies', 'septiesep': 'Septies',
            'octavus': 'Octavus', 'octies': 'Octies',
            'nonies': 'Nonies', 'noníes': 'Nonies', 'novies': 'Novies',
            'decies': 'Decies', 'decíes': 'Decies', 'decimus': 'Decimus',
            'undecies': 'Undecies', 'duodecies': 'Duodecies'
        }
        sufijo_ordinal = ordinales_normalizacion.get(sufijo_ordinal.lower(), sufijo_ordinal.capitalize())
    
    # Extrae número después del ordinal (ej: "Bis 2", "Bis 6", "Bis 7")
    numero_ordinal = None
    if sufijo_ordinal:
        match_num_ordinal = re.search(rf'{re.escape(sufijo_ordinal)}\s+(\d+)', mencion, re.IGNORECASE)
        if match_num_ordinal:
            numero_ordinal = int(match_num_ordinal.group(1))
    
    # Extrae letra sufijo (con o sin guión: "8o-A" o "272 A")
    letra_sufijo = None
    match_letra_guion = re.search(r'-([A-Z])\b', mencion)
    if match_letra_guion:
        letra_sufijo = match_letra_guion.group(1)
    elif not sufijo_ordinal:  # Solo busca letra sin guión si NO hay ordinal
        match_letra_espacio = re.search(r'\d+[°ºªo]?\s+([A-Z])\b', mencion)
        if match_letra_espacio:
            letra_sufijo = match_letra_espacio.group(1)
    
    # Extrae fracción numérica (ej: "23-1") - solo si no hay letra ni ordinal
    fraccion = None
    if not letra_sufijo and not sufijo_ordinal:
        match_fraccion = re.search(r'-(\d+)\b', mencion)
        fraccion = int(match_fraccion.group(1)) if match_fraccion else None
    
    return (numero_base, sufijo_ordinal, numero_ordinal, fraccion, letra_sufijo)


def validar_secuencia_articulos(articulos):
    """
    Valida que los artículos sigan una secuencia lógica.
    Detecta posibles errores como:
    - Artículos repetidos sin variación (falta Bis, -1, etc.)
    - Saltos en la numeración sin justificación
    
    Retorna la lista de artículos con advertencias agregadas.
    """
    if len(articulos) < 2:
        return articulos
    
    # Orden de ordinales latinos
    orden_ordinales = {
        None: 0,
        'Bis': 1, 'Ter': 2, 'Quáter': 3, 
        'Quintus': 4, 'Quinquies': 4, 'Quinties': 4,
        'Sextus': 5, 'Sexies': 5, 'Sexties': 5,
        'Septimus': 6, 'Septies': 6, 'Septiesep': 6,
        'Octavus': 7, 'Octies': 7,
        'Nonies': 8, 'Novies': 8,
        'Decies': 9, 'Decimus': 9,
        'Undecies': 10, 'Duodecies': 11
    }
    
    articulos_validados = []
    
    for i, articulo in enumerate(articulos):
        art_actual = parsear_articulo(articulo['Articulo'])
        advertencias = []
        
        if i > 0:
            art_anterior = parsear_articulo(articulos[i-1]['Articulo'])
            
            if art_actual and art_anterior:
                num_base_actual, suf_ord_actual, num_ord_actual, frac_actual, letra_actual = art_actual
                num_base_anterior, suf_ord_anterior, num_ord_anterior, frac_anterior, letra_anterior = art_anterior
                
                # CASO 1: Mismo número base
                if num_base_actual == num_base_anterior:
                    if suf_ord_actual == suf_ord_anterior:
                        if num_ord_actual == num_ord_anterior and frac_actual == frac_anterior and letra_actual == letra_anterior:
                            advertencias.append("⚠️ REPETIDO: Artículo duplicado sin variación")
                        elif num_ord_actual and num_ord_anterior:
                            if num_ord_actual != num_ord_anterior + 1:
                                advertencias.append(f"⚠️ SALTO: Esperaba '{articulos[i-1]['Articulo']}' → 'Artículo {num_base_actual} {suf_ord_actual} {num_ord_anterior + 1}'")
                    elif not suf_ord_anterior and suf_ord_actual:
                        pass  # Transición válida: Art 449 → Art 449 Bis
                    elif letra_actual and letra_anterior:
                        if ord(letra_actual) != ord(letra_anterior) + 1:
                            letra_esperada = chr(ord(letra_anterior) + 1)
                            advertencias.append(f"⚠️ SALTO: Esperaba 'Artículo {num_base_actual} {letra_esperada}'")
                    elif frac_actual and frac_anterior:
                        if frac_actual != frac_anterior + 1:
                            advertencias.append(f"⚠️ SALTO: Esperaba 'Artículo {num_base_actual}-{frac_anterior + 1}'")
                
                # CASO 2: Número base incrementa
                elif num_base_actual == num_base_anterior + 1:
                    pass  # Secuencia normal
                
                # CASO 3: Salto mayor en número base
                elif num_base_actual > num_base_anterior + 1:
                    advertencias.append(f"ℹ️ SALTO NUMÉRICO: De {num_base_anterior} a {num_base_actual}")
                
                # CASO 4: Número base menor
                else:
                    advertencias.append(f"⚠️ RETROCESO: Artículo {num_base_actual} después de {num_base_anterior}")
        
        # Agrega el artículo con sus advertencias
        articulo_validado = articulo.copy()
        if advertencias:
            articulo_validado['Advertencias'] = advertencias
        
        articulos_validados.append(articulo_validado)
    
    return articulos_validados


def limpiar_contenido_articulo(contenido):
    """
    Limpia el contenido del artículo eliminando elementos innecesarios.
    """
    if not contenido:
        return ""
    
    # Elimina saltos de línea excesivos
    contenido = re.sub(r'\n{3,}', '\n\n', contenido)
    
    # Elimina espacios múltiples
    contenido = re.sub(r' {2,}', ' ', contenido)
    
    # Elimina espacios al inicio de cada línea
    lineas = contenido.split('\n')
    lineas_limpias = [linea.strip() for linea in lineas if linea.strip()]
    contenido = '\n'.join(lineas_limpias)
    
    # Elimina caracteres especiales de formateo de PDF
    contenido = re.sub(r'[\x00-\x08\x0b-\x0c\x0e-\x1f]', '', contenido)
    
    return contenido.strip()

In [48]:
def recortar_pdf_headers_footers(ruta_pdf, crop_top_mm=20, crop_bottom_mm=20):
    """
    Recorta encabezados y pies de página de un PDF.
    
    Args:
        ruta_pdf: Ruta al archivo PDF original
        crop_top_mm: Milímetros a recortar desde arriba (encabezado)
        crop_bottom_mm: Milímetros a recortar desde abajo (pie de página)
    
    Returns:
        Ruta al archivo PDF temporal recortado
    """
    try:
        # Abrir el PDF
        doc = fitz.open(ruta_pdf)
        
        # Crear PDF temporal
        temp_pdf = tempfile.NamedTemporaryFile(delete=False, suffix='.pdf')
        temp_pdf_path = temp_pdf.name
        temp_pdf.close()
        
        # Crear nuevo PDF con páginas recortadas
        pdf_out = fitz.open()
        
        for page_num in range(len(doc)):
            page = doc[page_num]
            
            # Obtener dimensiones de la página
            rect = page.rect
            
            # Convertir mm a puntos (1 mm = 2.83465 puntos)
            crop_top_pts = crop_top_mm * 2.83465
            crop_bottom_pts = crop_bottom_mm * 2.83465
            
            # Definir el área a conservar (recortando arriba y abajo)
            new_rect = fitz.Rect(
                rect.x0,                    # izquierda (sin cambio)
                rect.y0 + crop_top_pts,     # arriba (recortar)
                rect.x1,                    # derecha (sin cambio)
                rect.y1 - crop_bottom_pts   # abajo (recortar)
            )
            
            # Aplicar recorte
            page.set_cropbox(new_rect)
            
            # Agregar página al nuevo PDF
            pdf_out.insert_pdf(doc, from_page=page_num, to_page=page_num)
        
        # Guardar PDF recortado
        pdf_out.save(temp_pdf_path)
        pdf_out.close()
        doc.close()
        
        return temp_pdf_path
    
    except Exception as e:
        print(f"Error al recortar PDF '{ruta_pdf}': {e}")
        return ruta_pdf  # Devolver PDF original si falla


In [49]:
def extraer_texto_pdf(ruta_pdf, recortar_headers=True, crop_top_mm=20, crop_bottom_mm=20):
    """
    Extrae el texto de un PDF usando Apache Tika.
    Opcionalmente recorta encabezados y pies de página antes de extraer.
    
    Args:
        ruta_pdf: Ruta al archivo PDF
        recortar_headers: Si True, recorta encabezados/pies antes de extraer
        crop_top_mm: Milímetros a recortar desde arriba
        crop_bottom_mm: Milímetros a recortar desde abajo
    """
    pdf_temp = None
    try:
        # Recortar PDF si se solicita
        if recortar_headers:
            pdf_temp = recortar_pdf_headers_footers(ruta_pdf, crop_top_mm, crop_bottom_mm)
            pdf_a_procesar = pdf_temp
        else:
            pdf_a_procesar = ruta_pdf
        
        # Extraer texto con Tika
        raw = parser.from_file(pdf_a_procesar)  
        texto = raw.get("content", "").strip()
        
        # Limpiar archivo temporal
        if pdf_temp and pdf_temp != ruta_pdf:
            try:
                os.unlink(pdf_temp)
            except:
                pass
        
        return texto
    
    except Exception as e:
        print(f"Error al procesar '{ruta_pdf}': {e}")
        # Limpiar archivo temporal en caso de error
        if pdf_temp and pdf_temp != ruta_pdf:
            try:
                os.unlink(pdf_temp)
            except:
                pass
        return None


In [50]:
def procesar_documentos(carpeta, 
                       recortar_pdf_headers=True, 
                       crop_top_mm=20, 
                       crop_bottom_mm=20):
    """ 
    Procesa PDFs y DOCX en una carpeta y extrae los artículos.
    
    Args:
        carpeta: Ruta a la carpeta con documentos
        recortar_pdf_headers: Si True, recorta encabezados/pies de PDFs antes de extraer
        crop_top_mm: Milímetros a recortar desde arriba en PDFs
        crop_bottom_mm: Milímetros a recortar desde abajo en PDFs
    """
    todos_articulos = []

    for archivo in os.listdir(carpeta):
        ruta = os.path.join(carpeta, archivo)

        if archivo.lower().endswith(".pdf"):
            print(f"Procesando PDF: {archivo} (recortando {crop_top_mm}mm arriba, {crop_bottom_mm}mm abajo)")
            texto = extraer_texto_pdf(ruta, 
                                     recortar_headers=recortar_pdf_headers,
                                     crop_top_mm=crop_top_mm,
                                     crop_bottom_mm=crop_bottom_mm)

        elif archivo.lower().endswith((".docx", ".doc")):
            print(f"Procesando DOCX: {archivo}")
            texto = extraer_texto_docx(ruta)

        else:
            continue  # Ignorar archivos que no sean PDF o DOCX

        if not texto:
            continue  # Saltar archivos con errores o vacíos

        # Obtener nombre del archivo sin extensión
        nombre_archivo = os.path.splitext(archivo)[0]

        # FLUJO MEJORADO:
        # 1. Normalizar
        texto = normaliza_texto(texto)
        
        # 2. Separar transitorios del texto principal
        texto_principal, texto_transitorios = separar_transitorios(texto)
        
        # 3. Quitar encabezados estructurales aislados del texto principal
        texto_principal = quitar_encabezados_conservador(texto_principal)
        
        # 4. Extraer artículos del texto principal (incluye limpieza individual)
        articulos = extraer_articulos(texto_principal)
        
        for art in articulos:
            art["Archivo"] = nombre_archivo

        todos_articulos.extend(articulos)
        
        # 5. Si hay transitorios, agregarlos como un artículo especial
        if texto_transitorios:
            # Limpiar el contenido de los transitorios (igual que artículos)
            texto_transitorios_limpio = limpiar_contenido_articulo(texto_transitorios)
            
            todos_articulos.append({
                "Archivo": nombre_archivo,
                "Articulo": "TRANSITORIOS",
                "Texto": texto_transitorios_limpio
            })

    return todos_articulos


In [51]:
if __name__ == '__main__':
    # Procesar documentos y extraer artículos
    carpeta = "/Users/alexa/Projects/cdmx_pipeline/data/01_input/docx_leyes/"
    
    # Parámetros de recorte para PDFs (ajusta según necesites)
    # crop_top_mm: milímetros a recortar desde arriba (encabezado)
    # crop_bottom_mm: milímetros a recortar desde abajo (pie de página)
    lista_articulos = procesar_documentos(
        carpeta, 
        recortar_pdf_headers=True,  # Cambiar a False para desactivar recorte
        crop_top_mm=20,             # Ajustar según el tamaño de encabezados
        crop_bottom_mm=20           # Ajustar según el tamaño de pies de página
    )
    
    # Crear DataFrame con columnas básicas
    df = pd.DataFrame(lista_articulos, columns=["Archivo", "Articulo", "Texto"])
    
    print(f"\nTotal de artículos extraídos: {len(df)}")
    print(f"Total de documentos únicos: {df['Archivo'].nunique()}")
    print("\nPrimeros 5 artículos:")
    print(df.head())
    
    # Verificar si hay artículos con texto muy corto (posible pérdida de contenido)
    articulos_cortos = df[df['Texto'].str.len() < 20]
    print(f"\nArtículos con texto < 20 caracteres: {len(articulos_cortos)}")
    if len(articulos_cortos) > 0:
        print("Ejemplos:")
        print(articulos_cortos.head(10))

    output_path = "/Users/alexa/Projects/cdmx_pipeline/data/01_input/extracted_articles.xlsx"
    df.to_excel(output_path, index=False)
    print(f"\nArchivo guardado en: {output_path}")


Procesando DOCX: RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA_CDMX.docx
Procesando DOCX: REGLAMENTODEINTEGRACIONFUNCIONAMIENTOYSESIONESDELOSCONSEJOSDISTRITALESDELINSTITUTOELECTORALDELDISTRITOFEDERAL3.docx
Procesando DOCX: RGTO_INTERIOR_DE_LA_UNIVERSIDAD_DE_LA_POLICIA_CDMX_2.5.docx
Procesando DOCX: REGLAMENTO_LEY_PROTECCION20_A_LA_SALUD_DE_LOS_NO_FUMADORES_EN_EL_DF_3.docx
Procesando DOCX: LEY_BANCO_ADN_PARA_USO_FORENSE_DE_LA_CIUDAD_DE_MEXICO_2.1.docx
Procesando DOCX: LEY_PROCESAL_ELECTORAL_DE_LA_CDMX_5.3.docx
Procesando DOCX: LEY_DE_MEJORA_REGULATORIA_PARA_LA_CIUDAD_DE_MEXICO_3.4.docx
Procesando PDF: REGLAMENTO_DE_LA_LEY_DE_EJECUCION_DE_SANCIONES_PENALES_Y_REINSERCION_SOCIAL_PARA_EL_DISTRITO_FEDERAL_1.pdf (recortando 20mm arriba, 20mm abajo)
Procesando DOCX: RGTO_LEY_DE_GESTION_INTEGRAL_DE_RIESGOS_Y_PROTECCION_CIVIL_5.docx
Procesando DOCX: LEY_PARA_LA_PREV_Y_TRAT_DE_LA_OBESIDAD_CDMX_3.docx
Procesando DOCX: LEY_DE_RESPONSABILIDADES_ADMINISTRATIVAS_DE_LA_CDMX_3.1.docx
Procesando DOCX: L

In [52]:
df.shape

(31037, 3)

In [53]:
df.head()

,Archivo,Articulo,Texto
0,RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA...,Artículo 1,Las disposiciones contenidas en este Reglament...
1,RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA...,Artículo 2,La aplicación del presente Reglamento correspo...
2,RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA...,Artículo 3,"Para los efectos del presente Reglamento, adem..."
3,RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA...,Artículo 4,Sin perjuicio de los principios que prevén los...
4,RGTO_DE_LA_LEY_DE_CENTROS_PENITENCIARIOS_DE_LA...,Artículo 5,El tratamiento para la persona privada de su l...
